In [187]:
path = "/Users/gleschen/Downloads/test_raw_file.txt"
file=open(path,'r')
print(file.readlines())

['*\\n *  or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES *\\{\'iot-device-simulator\': {\'desc\': \'The IoT Device Simulator solution is a Graphical User Interface (GUI) based engine designed to enable customers to get started quickly assessing AWS IoT services without an existing pool of devices. The IoT Device Simulator helps effortlessly create and simulate thousands of connected devices that are defined by the customer.\', \'app.js\': \'/*********************************************************************************************************************\\n *  Copyright 2016 Amazon.com, Inc. or its affiliates. All Rights Reserved.                                           *\\n *                                                                                                                    *\\n *  Licensed under the Amazon Software License (the "License"). You may not use this file except in compliance        *\\n 

In [229]:
path = "/Users/gleschen/Downloads/test_raw_file.txt"
file=open(path,'r')
lines=file.readlines()
cummulateString=''
for line in lines:
#print(newLine)
    for char in line:
# Exclude the non alphabitical data.
        if(char.isalpha()):
            cummulateString+=char
        elif(char.isspace()):
            cummulateString+=''
        elif(char.isupper()):
            char=char.lower()
            cummulateString+=char
        else:
            cummulateString+=' '
            cummulateString+=' '
print(cummulateString)
#file.close()
cummulateString+=' '

    n  orinthe  license  fileaccompanyingthisfile  Thisfileisdistributedonan  ASIS  BASIS  WITHOUTWARRANTIES        iot  device  simulator        desc      TheIoTDeviceSimulatorsolutionisaGraphicalUserInterface  GUI  basedenginedesignedtoenablecustomerstogetstartedquicklyassessingAWSIoTserviceswithoutanexistingpoolofdevices  TheIoTDeviceSimulatorhelpseffortlesslycreateandsimulatethousandsofconnecteddevicesthataredefinedbythecustomer        app  js                                                                                                                                                                                                                                                    n  Copyright        Amazon  com  Inc  oritsaffiliates  AllRightsReserved      n      n  LicensedundertheAmazonSoftwareLicense  the  License      Youmaynotusethisfileexceptincompliance    n  withtheLicense  AcopyoftheLicenseislocatedat    n      n  http      aws  amazon  com  asl      n  n  ORCONDITIONSOF

In [230]:
#import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [231]:
import sys
'''
def remove_stopwords(cummulateString):
        processed_word_list = []
        for word in cummulateString:
            word = word.lower() # in case they aren't all lower case
            if word not in stopwords.words("english"):
                processed_word_list.append(word)
        print (cummulateString)
        return processed_word_list
'''
#example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(cummulateString)  
filtered_sentence = [w for w in word_tokens if not w in stop_words]  
filtered_sentence = [] 
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
print(word_tokens) 
print(filtered_sentence) 
with open('cleaned_data.txt', 'w') as f:
    print(filtered_sentence, file=f)  
file.close()

['n', 'orinthe', 'license', 'fileaccompanyingthisfile', 'Thisfileisdistributedonan', 'ASIS', 'BASIS', 'WITHOUTWARRANTIES', 'iot', 'device', 'simulator', 'desc', 'TheIoTDeviceSimulatorsolutionisaGraphicalUserInterface', 'GUI', 'basedenginedesignedtoenablecustomerstogetstartedquicklyassessingAWSIoTserviceswithoutanexistingpoolofdevices', 'TheIoTDeviceSimulatorhelpseffortlesslycreateandsimulatethousandsofconnecteddevicesthataredefinedbythecustomer', 'app', 'js', 'n', 'Copyright', 'Amazon', 'com', 'Inc', 'oritsaffiliates', 'AllRightsReserved', 'n', 'n', 'LicensedundertheAmazonSoftwareLicense', 'the', 'License', 'Youmaynotusethisfileexceptincompliance', 'n', 'withtheLicense', 'AcopyoftheLicenseislocatedat', 'n', 'n', 'http', 'aws', 'amazon', 'com', 'asl', 'n', 'n', 'ORCONDITIONSOFANYKIND', 'expressorimplied', 'SeetheLicenseforthespecificlanguagegoverningpermissions', 'n', 'andlimitationsundertheLicense', 'n', 'n', 'n', 'n', 'authorSolutionBuilders', 'n', 'n', 'n', 'usestrict', 'n', 'nconstf

In [ ]:
'''
import mmap
with open('cleaned_data.txt') as f:
    s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    if s.find(b'IoT'or b'device' or b'automotive' or b'vehicle' or b'car') != -1:
        print("Microservice is classified as: Automotive")
'''

In [232]:
import pandas
df = pandas.read_csv('dictionary.csv')
print(df)

  Banking Automotive    Media  Fintech  Transportation  Law Food&Beverages  \
0    Bank        iot  netflix  fintech  transportation  law       beverage   

  Healthcare Education   Real Estate  Insurance       Energy  Retail  
0     health    school  construction  insurance  electricity  retail  


In [233]:
print(list(df.values))

[array(['Bank', 'iot', 'netflix', 'fintech', 'transportation', 'law',
       'beverage', 'health', 'school', 'construction', 'insurance',
       'electricity', 'retail'], dtype=object)]


In [212]:
"""
import csv
#reader = csv.DictReader(open("file2.csv"))
df = csv.DictReader(open("dictionary.csv"))
for raw in df:
    orderedDict = raw
    #print(raw)
    print (raw)
"""

OrderedDict([('Banking', 'Bank'), ('Automotive', 'iot'), ('Media', 'netflix'), ('Fintech', 'fintech'), ('Transportation', 'transportation'), ('Law', 'law'), ('Food&Beverages', 'beverage'), ('Healthcare', 'health'), ('Education', 'school'), ('Real Estate', 'construction'), ('Insurance', 'insurance'), ('Energy', 'electricity'), ('Retail', 'retail')])


In [234]:
print(orderedDict.values())

odict_values(['Bank', 'iot', 'netflix', 'fintech', 'transportation', 'law', 'beverage', 'health', 'school', 'construction', 'insurance', 'electricity', 'retail'])


In [235]:
import mmap
with open('cleaned_data.txt') as f:
    s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    dic = orderedDict.values()
    print(list(dic))
    d = list(dic)
    #if s.find(b'dic') != -1:
    print(s.find(bytes(d[1].encode('utf-8'))))
    #print("Microservice is classified as: Automotive")
    f.close()

['Bank', 'iot', 'netflix', 'fintech', 'transportation', 'law', 'beverage', 'health', 'school', 'construction', 'insurance', 'electricity', 'retail']
124


In [237]:
with open('cleaned_data.txt') as f:
    cleanedData=f.read()
    for col in df.columns:
        for indx in range(0,len(df.loc[:,col])):
          if df.loc[:,col][indx] in cleanedData:
            #print(key.d[0])
            #print([key for key in raw.keys()][0])
            print("This microservice is classified as:", col)
            #print("true")

This microservice is classified as: Banking
This microservice is classified as: Automotive
